In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# import ML
import lightgbm as lgb #gradient boosted decsion tree machine
from sklearn.model_selection import KFold, RandomizedSearchCV #cross validation
from sklearn.model_selection import train_test_split #split
from sklearn.metrics import r2_score, mean_squared_error #metrics

# set up plot styles
sns.set_theme()
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_context("talk")